In [2]:
#Version 3.9 de Python (Conda)

  Using cached absl_py-1.0.0-py3-none-any.whl (126 kB)
  Using cached cachetools-5.0.0-py3-none-any.whl (9.1 kB)
  Using cached Cartopy-0.18.0.tar.gz (14.4 MB)


    ERROR: Command errored out with exit status 1:
     command: 'C:\Users\Shounen\anaconda3\envs\tenv\python.exe' -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\Shounen\\AppData\\Local\\Temp\\pip-install-7v0lvo2m\\cartopy_3d2a0d4faf794352843444e5b5d6baaa\\setup.py'"'"'; __file__='"'"'C:\\Users\\Shounen\\AppData\\Local\\Temp\\pip-install-7v0lvo2m\\cartopy_3d2a0d4faf794352843444e5b5d6baaa\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base 'C:\Users\Shounen\AppData\Local\Temp\pip-pip-egg-info-bzd5rgf7'
         cwd: C:\Users\Shounen\AppData\Local\Temp\pip-install-7v0lvo2m\cartopy_3d2a0d4faf794352843444e5b5d6baaa\
    Complete output (3 lines):
    C:\Users\Shounen\AppData\Local\Temp\pip-install-7v0lvo2m\cartopy_3d2a0d4fa

In [1]:
from netCDF4 import Dataset, num2date
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cf
import tensorflow as tf
import pandas as pd
import re
from tensorflow.keras import datasets, layers, models
from sklearn.model_selection import train_test_split

ModuleNotFoundError: No module named 'netCDF4'

In [13]:
#El modelo solo considera en input_shape(x,x,1), el 1 se puede cambiar para abarcar mas canales de imagenes satelitales
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(110, 110, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10))

#-Visualizacion
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 108, 108, 32)      320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 54, 54, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 52, 52, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 26, 26, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 24, 24, 64)        36928     
                                                                 
 flatten (Flatten)           (None, 36864)             0

In [14]:
#Se le da un tensor de 4 dimensiones
#[0] =  dato de precipitacion
#[1] = Punto de la estacion (Longitud)
#[2] = Punto de la estacion (Latitud)
#[3] = El nombre del archivo que contiene la imagen satelital


def leerImagenArea(tensor):
    """
    Los archivos se deben encontrar en carpetas ordenadas : ../GOES/{producto}/{año}/{mes}/archivo.nc
    
    EJEMPLO : path_base + GOES/C8/2019/02/G16_C08_Cyl_20190210-1600.nc
    """
    #Se define por defecto el path base (Temporal)
    path_base  =  'C:/Users/Shounen/Desktop/Ciclo XI/Tesis 2'
    
    
    
    #El ancho y alto sera el margen que se dara desde el punto de origen (estacion)
    #Esta en grados (1 grado == 100Km) - (Temporal)
    ancho=1
    alto=1
    
    
    filename = tensor.numpy()[3].decode('UTF-8')
    origen = [float(tensor.numpy()[1].decode('UTF-8')),float(tensor.numpy()[2].decode('UTF-8'))]   
    
    try:
        ds = Dataset(filename)
    except:
        return -1
    
    # convierte el tiempo de formato numerico a formato fecha y hora de python
    #date = num2date(ds.variables['time'][:], ds.variables['time'].units, only_use_cftime_datetimes=False, only_use_python_datetimes=True)
    
    # convierte el formato de la variable de Int16 a Float32 y guarda el resultado
    field = ds.variables['CMI'][:].data.astype(np.float32)/100.0
    
    # obtiene las coordenadas de los pixeles
    lon = ds.variables['longitude'][:].data
    lat = ds.variables['latitude'][:].data    
    
    #Se define el margen para recortar la imagen satelital
    maxLon=origen[0]+ancho
    minLon=origen[0]-ancho
    maxLat=origen[1]+alto
    minLat=origen[1]-alto
    
    #Booleanos que ayudarán a buscar el margen
    altoMin = False
    altoMax = False
    
    
    #Inicializamos los "indices"
    lom = 0
    loM = 0
    lam = 0
    laM = 0
    
    """
    Tener en cuenta que el arreglo de longitudes (lon) esta ordenado de manera creciente,
    mientras que el de latitudes (lat) esta de manera decreciente
    """    
    for i in range(0,len(lon)):
        if lon[i]>=minLon and not altoMin:
            altoMin = True
            lom = i
        if lon[i]<=maxLon:
            loM = i
                
    for j in range(0,len(lat)):
        if lat[j]>=minLat:    
            laM = j
        if lat[j]<=maxLat and not altoMax:
            altoMax = True
            lam = j           
    
    #return lon[lom:loM],lat[lam:laM], field[lam:laM,lom:loM]    
    return field[lam:laM,lom:loM]
       

In [15]:
#Se obtiene los datos de precipitaciones desde el archivo CSV
datosFile = 'C:/Users/Shounen/Desktop/Ciclo XI/Tesis 2/SENAMHI/X4AD000C8.csv'

#Se lee desde la fila 44800, pues dede ahí comienza los datos del año 2019
pdata = pd.read_csv(datosFile, na_values = "0", skiprows = 44800)
dList = pdata.values.tolist()


#-Visualizacion
print(len(dList))
print(dList[0])

20912
[44800, 'OCROS', -77.39676, -10.40312, nan, nan, "['2019', '02', '10']", 'C:/Users/Shounen/Desktop/Ciclo XI/Tesis 2/GOES/C8/2019/02/G16_C08_Cyl_20190210-1600.nc']


In [16]:
#Se recorta la lista de datos para hacer pruebas
dList = dList[0:20]

In [17]:
#Con los datos del CSV, se obtiene solo lo necesario para el entrenamiento
#x[5] = dato de precipitacion
#x[2] = Estacion (Longitud)
#x[3] = Estacion (Latitud)
#x[7] = Nombre del archivo de la imagen satelital
imagenT = []
for x in dList: 
    
    if pd.isna(x[5]):
        datoTemp = 0.0
    else:
        datoTemp = x[5]
    imagenT.append([datoTemp,x[2],x[3],x[7]])

#Se convierte en numpy.array la lista
imagenT = np.array(imagenT)
imagenT  = np.nan_to_num(imagenT)


#-Visualizacion
print(imagenT.shape)
print(imagenT[0])

(20, 4)
['0.0' '-77.39676' '-10.40312'
 'C:/Users/Shounen/Desktop/Ciclo XI/Tesis 2/GOES/C8/2019/02/G16_C08_Cyl_20190210-1600.nc']


In [18]:
#Se separa el dataset (datos entrenamiento y validacion)
dataset = tf.data.Dataset.from_tensor_slices(imagenT)

train_size = int(len(imagenT)*0.8)
train_ds = dataset.take(train_size)
test_ds = dataset.skip(train_size)


#-Visualizacion
for d in train_ds.take(1):
    print(d.numpy())

[b'0.0' b'-77.39676' b'-10.40312'
 b'C:/Users/Shounen/Desktop/Ciclo XI/Tesis 2/GOES/C8/2019/02/G16_C08_Cyl_20190210-1600.nc']


In [19]:
#Servira para obtener el arreglo que representa la imagen satelital (recortada) usando el nombre del archivo
def _parse_function(dato):
    #Se llama a la funcion leerImagenArea con el tensor, retorna el arreglo que representa la imagen
    y = tf.py_function(func=leerImagenArea, inp=[dato], Tout=tf.float32)   
    
    #Retorna el arreglo (imagen) , dato de precipitacion
    return  y, float(dato[0])

In [20]:
#Probamos que funcione correctamente la funcion de parseo
for xTemp in dataset.take(1):
    xImg,xDato = _parse_function(xTemp)
    
#-Visualizacion
print(xImg)
print(xDato)

tf.Tensor(
[[244.35 244.3  244.35 ... 242.7  242.87 242.95]
 [244.3  244.51 244.43 ... 242.74 242.99 242.99]
 [244.43 244.51 244.56 ... 242.74 242.91 243.08]
 ...
 [226.43 227.61 228.04 ... 239.23 239.4  239.32]
 [226.43 227.61 228.04 ... 239.32 239.28 239.23]
 [222.59 221.66 223.31 ... 239.23 239.15 239.32]], shape=(110, 110), dtype=float32)
0.0


In [21]:
#Se les aplica la funcion de parseo al dataset de entrenamiento y validacion
train_dsX = train_ds.map(_parse_function)
test_dsX = test_ds.map(_parse_function)

#-Visualizacion
print(train_dsX)
print(test_dsX)

<MapDataset element_spec=(TensorSpec(shape=<unknown>, dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.float32, name=None))>
<MapDataset element_spec=(TensorSpec(shape=<unknown>, dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.float32, name=None))>


In [22]:
#Definimos el prefetch en los datos de validacion y entrenamiento
#AUTOTUNE = tf.data.AUTOTUNE

#train_dsX = train_dsX.cache().prefetch(buffer_size=AUTOTUNE)
#test_dsX = test_dsX.cache().prefetch(buffer_size=AUTOTUNE)

In [23]:
#Entrenamos el modelo
model.compile(optimizer='adam',
              loss=tf.keras.losses.MeanAbsoluteError(),
              metrics=['accuracy'])

history = model.fit(train_dsX, epochs=1,validation_data=test_dsX)

ValueError: in user code:

    File "C:\Users\Shounen\anaconda3\envs\Tesis\lib\site-packages\keras\engine\training.py", line 1021, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\Shounen\anaconda3\envs\Tesis\lib\site-packages\keras\engine\training.py", line 1010, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\Shounen\anaconda3\envs\Tesis\lib\site-packages\keras\engine\training.py", line 1000, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\Shounen\anaconda3\envs\Tesis\lib\site-packages\keras\engine\training.py", line 860, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "C:\Users\Shounen\anaconda3\envs\Tesis\lib\site-packages\keras\engine\training.py", line 918, in compute_loss
        return self.compiled_loss(
    File "C:\Users\Shounen\anaconda3\envs\Tesis\lib\site-packages\keras\engine\compile_utils.py", line 212, in _

In [ ]:
#Vemos lso resultados
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')